In [2]:
######################
# Process MIDI Files #
######################

from music21 import converter
from music21 import instrument
from music21 import chord
from music21 import note
import glob
import json
import os

# VARS
DATA_GLOB = "data/pokemon/PkmRB/*.mid"
MODEL_DIR = "model/pokemon/PkmRB-V1"
NOTES_FILE = "%s/%s" % (MODEL_DIR, "notes.json")

NOTES_BY_FILE = []
NOTES = set()

for input_file in glob.glob(DATA_GLOB):
    print("Ingesting... %s" % input_file)
    
    midi = converter.parse(input_file)
    parts = instrument.partitionByInstrument(midi)
    if parts:
        notes_to_parse = parts.parts[0].recurse()
    else:
        notes_to_parse = midi.flat.notes
    
    file_notes = []
    for element in notes_to_parse:
        if isinstance(element, note.Note):
            file_notes.append(str(element.pitch))
        elif isinstance(element, chord.Chord):
            file_notes.append('.'.join(str(n) for n in element.normalOrder))
    NOTES_BY_FILE.append(file_notes)
    NOTES.update(set(file_notes))

NOTES = sorted(NOTES)

if not os.path.exists(os.path.dirname(NOTES_FILE)):
    os.makedirs(os.path.dirname(NOTES_FILE))
with open(NOTES_FILE, 'w') as note_file:
    json.dump(NOTES, note_file)

Ingesting... data/pokemon/PkmRB/PkmRB-Battle3.mid
<music21.stream.iterator.StreamIterator for Score:0x7f7f2d9e8358 @:0>
Ingesting... data/pokemon/PkmRB/PkmRB-Evolving.mid
<music21.stream.iterator.StreamIterator for Score:0x7f7f2d938d30 @:0>
Ingesting... data/pokemon/PkmRB/PkmRB-Celadon.mid
<music21.stream.iterator.StreamIterator for Score:0x7f7f2c689e80 @:0>
Ingesting... data/pokemon/PkmRB/PkmRB-Victory1.mid
<music21.stream.iterator.StreamIterator for Score:0x7f7f2d59f438 @:0>
Ingesting... data/pokemon/PkmRB/PkmRB-10.mid
<music21.stream.iterator.StreamIterator for Score:0x7f7f2dc75a58 @:0>
Ingesting... data/pokemon/PkmRB/PkmRB-Pallet.mid
<music21.stream.iterator.StreamIterator for Score:0x7f7f2dabf320 @:0>
Ingesting... data/pokemon/PkmRB/PkmRB-Caught.mid
<music21.stream.iterator.StreamIterator for Score:0x7f7f2dd75d68 @:0>
Ingesting... data/pokemon/PkmRB/PkmRB-Cinnabar.mid
<music21.stream.iterator.StreamIterator for Score:0x7f7f2dd75e10 @:0>
Ingesting... data/pokemon/PkmRB/PkmRB-Battle

In [2]:
########################
# Build Note Sequences #
########################

import numpy
from keras.utils import np_utils

NOTES_FILE = None
SEQUENCE_LENGTH = 100
X = []
y = []

# Load Notes from disk if desired
if NOTES_FILE:
    with open(NOTES_FILE, 'r') as notes_file:
        NOTES = json.loads(notes_file.read())

NOTE_TO_INT = dict((note, number) for number, note in enumerate(NOTES))
INT_TO_NOTE = dict((number, note) for number, note in enumerate(NOTES))

for notes in NOTES_BY_FILE:
    for i in range(0, len(notes) - SEQUENCE_LENGTH, 1):
        input_sequence = notes[i:i + SEQUENCE_LENGTH] # N Input Notes
        output_sequence = notes[i + SEQUENCE_LENGTH] # 1 Output Note
        
        X.append([NOTE_TO_INT[c] for c in input_sequence])
        y.append(NOTE_TO_INT[output_sequence])
        
N_SEQUENCES = len(X)
N_VOCAB = len(NOTES)

print("Num Sequences: %d" % N_SEQUENCES)
print("Num Vocab: %d" % N_VOCAB)

X = numpy.reshape(X, (N_SEQUENCES, SEQUENCE_LENGTH, 1)) / N_VOCAB
y = np_utils.to_categorical(y)

/home/bbamsch/.virtualenvs/cmpe257/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Num Sequences: 24664
Num Vocab: 56


In [3]:
#######################
# Construct the Model #
#######################

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.callbacks import ModelCheckpoint

CHECKPOINT_FILE = None

model = Sequential()
model.add(LSTM(
    512,
    input_shape=(X.shape[1], X.shape[2]),
    return_sequences=True
))
model.add(Dropout(0.3))
model.add(LSTM(512))
model.add(Dropout(0.3))
model.add(Dense(N_VOCAB))
model.add(Activation('softmax'))
if (CHECKPOINT_FILE):
    model.load_weights(CHECKPOINT_FILE)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
###################
# Train the Model #
###################

from keras.callbacks import ModelCheckpoint

filepath = "%s/%s" % (MODEL_DIR, 'LSTM512-weights-improvement-{epoch:02d}-{loss:.4f}.hdf5')
checkpoint = ModelCheckpoint(filepath, monitor='loss', save_best_only=True)
callbacks_list = [checkpoint]

model.fit(X, y, epochs=500, batch_size=128, callbacks=callbacks_list)

Epoch 1/500
24664/24664 [==============================] - 72s 3ms/step - loss: 3.2722
Epoch 2/500
24664/24664 [==============================] - 76s 3ms/step - loss: 3.2233
Epoch 3/500
24664/24664 [==============================] - 77s 3ms/step - loss: 3.1771
Epoch 4/500
20096/24664 [=======================>......] - ETA: 14s - loss: 3.1319